In [1]:
import os
import sys
import numpy as np
from random import shuffle
import wavio
import wave
import subprocess
import time
import argparse
import random
import pathlib

In [2]:
sys.path.append("../../src/")
sys.path.append("../../")

In [3]:
from common.tlopts import display_info;
import common.utils as U;
from Libs.SharedLibs import *

In [4]:
from Libs.SharedLibs import getFileList;
from Libs.datetime_util import genDataTimeStr;

In [5]:
# wav_src_dir = "../../datasets/CurrentUse/wav_files/"
wav_root_src_dir = "../../datasets/CurrentUse/wav_files/Single_Fold/"
version_string = "version7_madeinhome"

## acdnet original dataset processing codes

In [7]:
# def convert_sr(src_path, dst_path, sr):
#     print('* {} -> {}'.format(src_path, dst_path))
#     if not os.path.exists(dst_path):
#         os.mkdir(dst_path);
#     for src_file in sorted(glob.glob(os.path.join(src_path, '*.wav'))):
#         dst_file = src_file.replace(src_path, dst_path);
#         subprocess.call('ffmpeg -i {} -ac 1 -ar {} -loglevel error -y {}'.format(
#             src_file, sr, dst_file), shell=True);

In [8]:
# def preprocess_dataset(to_process_dir):
#     wavlst = getFileList(to_process_dir);
#     save_dir = os.path.join(to_process_dir, '20K');
#     if not os.path.exists(save_dir):
#         os.mkdir(save_dir)
#     sr = 20000;
#     for src_file in wavlst:
#         print(f"original scc_file:{src_file}")
#         wav_name = os.path.basename(src_file)[:-4];
#         wav_name = "{}_20K.wav".format(wav_name);
#         save_path = os.path.join(save_dir, wav_name)
#         print(f"save dir:{save_path}")
#         framerate = 0;
#         with wave.open(src_file, 'rb') as f:
#             framerate = f.getframerate();
#         if framerate != 20000:
#             subprocess.call('ffmpeg -i {} -ac 1 -ar {} -loglevel error -y {}'.format(
#                 src_file, sr, save_path), shell=True);
#             print(f"converted {src_file} sampling rate from {framerate} to 20K")

## Generating Single-Fold Training DataSet

In [11]:
# def create_base_train_dataset(upper_level_dir=None, fold_dirs=None, p_classes=None, n_classes=None, export_path=None):
def create_base_train_dataset(upper_level_dir=None, p_classes=None, n_classes=None, export_path=None):
    total_counter = 0;
    p_counter = 0;
    n_counter = 0;
    train_dataset = {};
    dict_key = 'fold1';
    # for fold in fold_dirs:
    train_dataset[dict_key] = {}
    train_sounds = []
    train_labels = []
    for t in p_classes:
    #Dealing with positive wav files
        p_current_dir = os.path.join(upper_level_dir,'positive', t);
        print(f"work on dir:{p_current_dir}");
        lbl = t[t.rfind('_')+1:]
        tmp_list = getFileList(p_current_dir)
        for f in tmp_list:
            sound = wavio.read(f).data.T[0]
            start = sound.nonzero()[0].min()
            end = sound.nonzero()[0].max()
            sound = sound[start: end + 1]  # Remove silent sections
            train_sounds.append(sound)
            train_labels.append(lbl)
            total_counter += 1;
            p_counter += 1;
            
    # n_classes_root_dir = os.path.join(upper_level_dir,'negative')
    for c in n_classes:
        n_current_dir = os.path.join(upper_level_dir,'negative',c)
        print(f"work on dir:{n_current_dir}");
        n_lbl = 99;#c[:c.find('_')]
        tmp_list2 = getFileList(n_current_dir)
        for f in tmp_list2:
            sound = wavio.read(f).data.T[0]
            start = sound.nonzero()[0].min()
            end = sound.nonzero()[0].max()
            sound = sound[start: end + 1]  # Remove silent sections
            train_sounds.append(sound)
            train_labels.append(n_lbl)
            total_counter += 1;
            n_counter += 1;

    train_dataset[dict_key]['sounds'] = train_sounds
    train_dataset[dict_key]['labels'] = train_labels
    np.savez(export_path, **train_dataset)
    print(f"Training Data is generated and save at {export_path}")
    print(f"Total wav files for trainset is {total_counter}");
    print(f"Total positive wav files for trainset is {p_counter}");
    print(f"Total negative wav files for trainset is {n_counter}");

In [12]:
# p_classes = ["alarm_52","help_mandrain_71"]
def main():
    upper_level_dir = "../../datasets/CurrentUse/wav_files/Single_Fold/train/"
    save_dir = os.path.join("../CurrentUse/generated_datasets/train/",version_string);
    if not pathlib.Path(save_dir).is_dir():
        os.makedirs(save_dir);
    output_path = os.path.join(save_dir,"single_fold_train_{}.npz".format(genDataTimeStr()));
    p_dirs = ['alarm_52','refined_moan_wail_edited_20240507_56','help_eng_71'];#[f for f in os.listdir(upper_level_dir+"positive")]
    n_dirs = [f for f in os.listdir(upper_level_dir+"negative")]
    print(f"positive classes:{p_dirs}")
    print("negative classes:")
    count1 = 0
    for c in n_dirs:
        count1 += 1
        print(f"{count1}:{c}")
    dataset = create_base_train_dataset(upper_level_dir=upper_level_dir, p_classes=p_dirs, n_classes=n_dirs, export_path=output_path);

In [15]:
main()

positive classes:['alarm_52', 'refined_moan_wail_edited_20240507_56', 'help_eng_71']
negative classes:
1:99_other_sounds
2:17_pouring_water_esc50
3:25_footsteps_esc50
4:30_door_wood_knock_esc50
5:28_snoring_esc50
6:10_rain_esc50
7:39_glass_breaking_esc50
8:22_clap_esc50
9:44_engine_esc50
10:0_dog_esc50
11:14_chirping_birds_esc50
12:33_door_wood_creaks_esc50
13:18_toilet_flush_esc50
14:24_coughing_esc50
15:27_brushing_teeth_esc50
16:26_laughing_esc50
17:21_sneezing_esc50
18:35_wash_machine_esc50
19:29_drinking_sipping_esc50
20:43_car_horn_esc50
21:5_cat_esc50
22:15_water_drop_esc50
23:36_vacuum_cleaner_esc50
24:19_thunderstorm_esc50
work on dir:../../datasets/CurrentUse/wav_files/Single_Fold/train/positive/alarm_52
work on dir:../../datasets/CurrentUse/wav_files/Single_Fold/train/positive/refined_moan_wail_edited_20240507_56
work on dir:../../datasets/CurrentUse/wav_files/Single_Fold/train/positive/help_eng_71
work on dir:../../datasets/CurrentUse/wav_files/Single_Fold/train/negative/99

In [17]:
ds = np.load("../CurrentUse/generated_datasets/train/version7_madeinhome/single_fold_train_20240519222746.npz", allow_pickle=True);
print(ds['fold1'].item()['labels'])

['52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52

## Generating Validation DataSet

In [18]:
class ValGenerator():
    #Generates data for Keras
    def __init__(self, samples, labels, options):
        random.seed(42);
        #Initialization
        print(len(samples));
        self.data = [(samples[i], labels[i]) for i in range (0, len(samples))];
        self.opt = options;
        self.batch_size = len(samples);#88;#options.batchSize // options.nCrops;
        print(f"batch_size:{self.batch_size}");
        self.preprocess_funcs = self.preprocess_setup();
        self.map_dict= {
            '52':1, #alarm
            '56':2, #moaning
            '71':3, #help(english)
            '99':4, #other_sounds 
        };

    def get_data(self):
        #Generate one batch of data
        x, y = self.generate();
        x = np.expand_dims(x, axis=1)
        x = np.expand_dims(x, axis=3)
        # print(x.shape);
        # print(y.shape);
        return x, y

    def generate(self):
        #Generates data containing batch_size samples
        sounds = [];
        labels = [];
        indexes = None;
        for i in range(self.batch_size):
            sound, target = self.data[i];
            target = self.map_dict[str(target)] - 1;
            sound = self.preprocess(sound).astype(np.float32)
            # print(sound)
            label = np.zeros((self.opt.nCrops, self.opt.nClasses));
            label[:,target] = 1;
            print(f"nCrops:{self.opt.nCrops}, nClasses:{self.opt.nClasses}")
            sounds.append(sound);
            labels.append(label);
        """
        #dtype="object" for ValueError: setting an array element with a sequence. 
        The requested array has an inhomogeneous shape after 1 dimensions. 
        The detected shape was (58,) + inhomogeneous part.
        """
        sounds = np.asarray(sounds,dtype="object")
        # expand_sounds = np.expand_dims(np.asarray(sounds,dtype="object"),axis=1); 
        labels = np.asarray(labels);
        # print(f"shape of sounds:{expand_sounds.shape}")
        sounds = sounds.reshape(sounds.shape[0]*sounds.shape[1], sounds.shape[2]);
        labels = labels.reshape(labels.shape[0]*labels.shape[1], labels.shape[2]);
        return sounds, labels;

    def preprocess_setup(self):
        funcs = []
        funcs += [U.padding(self.opt.inputLength // 2),
                  U.normalize(32768.0),
                  U.multi_crop(self.opt.inputLength, 2)] # we use single crop here.

        return funcs

    def preprocess(self, sound):
        for f in self.preprocess_funcs:
            sound = f(sound)

        return sound;

In [20]:
def getOpts():
    parser = argparse.ArgumentParser(description='Transfer Learning for ACDNet');
    parser.add_argument('--netType', default='ACDNet_TL_Model_Extend',  required=False);
    parser.add_argument('--data', default='../datasets/processed/',  required=False);
    parser.add_argument('--dataset', required=False, default='uec_iot', choices=['10']);
    parser.add_argument('--BC', default=True, action='store_true', help='BC learning');
    parser.add_argument('--strongAugment', default=True,  action='store_true', help='Add scale and gain augmentation');
    #在ipynb中，不能使用parser.parse，要改用parser.parse_known_args()
    opt, unknown = parser.parse_known_args();

    """
    current best setting for accuracy: 96.5
    opt.batchSize = 32;
    opt.LR = 0.1;
    opt.weightDecay = 5e-3;
    opt.momentum = 0.09;
    opt.schedule = [0.3, 0.5, 0.9];
    """
    #Leqarning settings
    opt.batchSize = 64;
    opt.LR = 0.1;
    opt.weightDecay = 5e-4;#1e-2;#5e-3;#5e-4;
    opt.momentum = 0.9;
    opt.nEpochs = 800;
    opt.schedule = [0.3, 0.5, 0.8];
    opt.warmup = 10;
    opt.device = 'cpu';
    # if torch.backends.mps.is_available():
    #     opt.device="mps"; #for apple m2 gpu
    # elif torch.cuda.is_available():
    #     opt.device="cuda:0"; #for nVidia gpu
    # else:
    #     opt.device="cpu"
    print(f"***Use device:{opt.device}");
    # opt.device = torch.device("cuda:0" if  else "cpu");
    #Basic Net Settings
    opt.nClasses = 4#50;
    opt.nFolds = 1;
    opt.splits = [i for i in range(1, opt.nFolds + 1)];
    opt.sr = 20000;
    opt.inputLength = 30225;
    #Test data
    opt.nCrops = 2;
    opt.TLAcdnetConfig = [8,64,32,64,64,128,128,256,256,512,512,2];
    return opt

In [23]:
def create_val_dataset_src_npy(dict_key='fold1', upper_level_dir=None, p_classes=None, n_classes=None, export_path=None):
    train_dataset = {};
    # dict_key = 'fold1';
    # for fold in fold_dirs:
    train_dataset[dict_key] = {}
    train_sounds = []
    train_labels = []
    total_counter = 0;
    p_counter = 0;
    n_counter = 0;
    for t in p_classes:
    #Dealing with positive wav files
        p_current_dir = os.path.join(upper_level_dir,'positive', t);
        print(f"work on dir:{p_current_dir}");
        lbl = t[t.rfind('_')+1:]
        tmp_list = getFileList(p_current_dir)
        print(f"class:{t}, item number:{len(tmp_list)}")
        for f in tmp_list:
            sound = wavio.read(f).data.T[0]
            start = sound.nonzero()[0].min()
            end = sound.nonzero()[0].max()
            sound = sound[start: end + 1]  # Remove silent sections
            train_sounds.append(sound)
            train_labels.append(lbl)
            total_counter += 1;
            p_counter += 1;
    for c in n_classes:
        n_current_dir = os.path.join(upper_level_dir,'negative',c)
        print(f"work on dir:{n_current_dir}");
        n_lbl = 99;#c[:c.find('_')]
        tmp_list2 = getFileList(n_current_dir)
        print(f"class:{c}, item number:{len(tmp_list2)}")
        for f in tmp_list2:
            sound = wavio.read(f).data.T[0]
            start = sound.nonzero()[0].min()
            end = sound.nonzero()[0].max()
            sound = sound[start: end + 1]  # Remove silent sections
            train_sounds.append(sound)
            train_labels.append(n_lbl)
            total_counter += 1;
            n_counter += 1;

    train_dataset[dict_key]['sounds'] = train_sounds
    train_dataset[dict_key]['labels'] = train_labels
    np.savez(export_path, **train_dataset)
    print(f"Test Data is generated and save at {export_path}")
    print(f"Total wav files for trainset is {total_counter}");
    print(f"Total positive wav files for trainset is {p_counter}");
    print(f"Total negative wav files for trainset is {n_counter}");

In [24]:
def val_main1():
    val_upper_level_dir = "../CurrentUse/wav_files/Single_Fold/val/"
    p_dirs = ['alarm_52','moaning_56','help_eng_71'];#[f for f in os.listdir(upper_level_dir+"positive")]
    n_dirs = [f for f in os.listdir(val_upper_level_dir+"negative")]
    save_path = os.path.join("../CurrentUse/generated_datasets/val/",version_string);
    if not pathlib.Path(save_path).is_dir():
        os.makedirs(save_path)
    output_fullname = os.path.join("../CurrentUse/generated_datasets/val/", version_string, "single_fold_val_src_{}.npz".format(genDataTimeStr()));
    print(f"positive classes:{p_dirs}")
    print("negative directories:")
    count1 = 0
    for c in n_dirs:
        count1 += 1
        print(f"{count1}:{c}")
    create_val_dataset_src_npy(upper_level_dir=val_upper_level_dir,p_classes=p_dirs,n_classes=n_dirs,export_path=output_fullname);

In [25]:
val_main1()

positive classes:['alarm_52', 'moaning_56', 'help_eng_71']
negative directories:
1:17_pouring_water_esc50
2:25_footsteps_esc50
3:30_door_wood_knock_esc50
4:28_snoring_esc50
5:10_rain_esc50
6:39_glass_breaking_esc50
7:22_clap_esc50
8:44_engine_esc50
9:0_dog_esc50
10:14_chirping_birds_esc50
11:33_door_wood_creaks_esc50
12:18_toilet_flush_esc50
13:24_coughing_esc50
14:27_brushing_teeth_esc50
15:26_laughing_esc50
16:21_sneezing_esc50
17:35_wash_machine_esc50
18:29_drinking_sipping_esc50
19:43_car_horn_esc50
20:5_cat_esc50
21:15_water_drop_esc50
22:36_vacuum_cleaner_esc50
23:19_thunderstorm_esc50
work on dir:../CurrentUse/wav_files/Single_Fold/val/positive/alarm_52
class:alarm_52, item number:44
work on dir:../CurrentUse/wav_files/Single_Fold/val/positive/moaning_56
class:moaning_56, item number:78
work on dir:../CurrentUse/wav_files/Single_Fold/val/positive/help_eng_71
class:help_eng_71, item number:63
work on dir:../CurrentUse/wav_files/Single_Fold/val/negative/17_pouring_water_esc50
clas

In [9]:
val_dataset = np.load('../../../uec_iot_ai_models_datasets/generated_datasets/val/version7_madeinhome/single_fold_val_src_20240519223131.npz',allow_pickle=True);

In [10]:
# print(len(val_dataset['fold1'].item()['labels']))
print(val_dataset['fold1'])

{'sounds': [array([13945, 15969,  -499, ..., -5408,  1119, 10024], dtype=int16), array([9945, 9139, 7566, ..., 6529, 8252, 9319], dtype=int16), array([    -1,      2,     -2, ...,  -8859, -12565, -15248], dtype=int16), array([   3,  -23,  -35, ..., 3298, 1500, 1342], dtype=int16), array([    4,    10,     4, ..., -3255, -3170, -2741], dtype=int16), array([ 900,  578, 1195, ...,    0,   -1,    1], dtype=int16), array([  -50,  -110,   -12, ..., 22527, 22111, -2698], dtype=int16), array([ 2494,  4458,  4846, ...,   518,  5809, 14633], dtype=int16), array([-218, -288, -225, ..., -328, -396, -205], dtype=int16), array([ -9442, -15684, -15069, ..., -18935, -21549,  13639], dtype=int16), array([1851, 3008, 2388, ...,  580,  624,  424], dtype=int16), array([ -2944,  -4796,  -4230, ...,   5683, -25563, -32768], dtype=int16), array([-1824,   738,  5389, ...,   927,  -607,   279], dtype=int16), array([-10148, -32767, -31492, ..., -28278, -18295,  12719], dtype=int16), array([-3308, -3324, -3352, 

In [31]:
def create_test_compress_npz(val_src_sounds=None, val_src_labels=None, export_path=None):
    opt = getOpts();#opts.parse();
    # display_info(opt);
    # opt.batchSize=88;
    opt.nCrops = 2;
    opt.nClasses= 4;
    opt.sr = 20000;
    opt.inputLength = 30225;
    val_sounds = [];
    val_labels = [];
   
    start_time = time.perf_counter();
    val_sounds.extend(val_src_sounds);
    val_labels.extend(val_src_labels);
    print(f"len of val_sounds:{len(val_sounds)}, len of val_labels:{len(val_labels)}")
    
    valGen = ValGenerator(val_src_sounds, val_src_labels, opt);
    valX, valY = valGen.get_data();

    np.savez_compressed(export_path, x=valX, y=valY);
    print('compressed npz generated with\n  shape x:{}\n  y:{}\n  took {:.2f} secs'.format(valX.shape, valY.shape, time.perf_counter()-start_time));
    sys.stdout.flush();

In [32]:
src_val_data_npz = "../CurrentUse/generated_datasets/val/version7_madeinhome/single_fold_val_src_20240519223131.npz"
val_data = np.load(src_val_data_npz, allow_pickle=True);
dest_path = "../CurrentUse/generated_datasets/val/{}/final_single_val_{}.npz".format(version_string,genDataTimeStr());
#call to create validation set
print(f"dest_path is {dest_path}")
_sounds = val_data['fold1'].item()['sounds']
_labels = val_data['fold1'].item()['labels']
create_test_compress_npz(_sounds,_labels,dest_path)

dest_path is ../CurrentUse/generated_datasets/val/version7_madeinhome/final_single_val_20240519223409.npz
***Use device:cpu
len of val_sounds:327, len of val_labels:327
327
batch_size:327
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nClasses:4
nCrops:2, nCla

In [26]:
val_data = np.load("../CurrentUse/generated_datasets/val/version6_office/final_single_val_20240509144233.npz",allow_pickle=True)

In [27]:
print(val_data['y'])

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [14]:
# test_list = [52 for _ in range(87)]

In [15]:
# print(test_list)

In [16]:
# print(len(test_list))

In [8]:
# test_lbl = 'alarm_52'
# print(test_lbl[test_lbl.rfind('_')+1:])

In [7]:
# test_lbl2 = '12_rainfall'
# print(test_lbl2[:test_lbl2.find('_')])